In [ ]:
# =========================
# 0) CONFIG (Formulaire)
# =========================
# Exécute cette cellule, ajuste les paramètres via le formulaire, puis clique "Appliquer".
# Les variables de config seront ensuite utilisées par le reste du notebook.

!pip -q install -U yt-dlp gdown

import os, re, json, pathlib
from IPython.display import display, Markdown, clear_output
import ipywidgets as widgets

# ---- Valeurs par défaut (tu peux les changer ici une fois, le formulaire s'en sert) ----
DEFAULTS = dict(
    # Source vidéo
    source_type="yt-dlp",  # "yt-dlp" ou "drive_share"
    video_url="",
    drive_share_url="",

    # yt-dlp
    ytdlp_format="bv*+ba/b",

    # Mode
    conversion_mode="preview",  # "preview" ou "full"

    # Sorties
    output_name_preview="preview_sbs.mp4",
    output_name_full="full_sbs.mp4",

    # Paramètres conversion (DepthAnything -> SBS)
    encoder="vits",          # vits|vitb|vitl|vitg (selon ton script)
    sbs_w=2560,
    sbs_h=1440,
    max_shift=24,
    alpha=0.90,
    input_size=518,
    batch=52,

    # Preview
    preview_interval=120,   # 1 frame sur N

    # Options
    keep_audio=True,
    crf=18,
    preset="slow",

    # Drive (optionnel)
    use_drive=False,          # ultrafast|veryfast|faster|fast|medium|slow|slower|veryslow
)

# ---- Widgets ----
source_type = widgets.ToggleButtons(
    options=[("Télécharger via yt-dlp", "yt-dlp"), ("Lien Drive partagé", "drive_share")],
    value=DEFAULTS["source_type"],
    description="Source:",
)

video_url = widgets.Text(
    value=DEFAULTS["video_url"],
    placeholder="https://... (YouTube, etc.)",
    description="URL yt-dlp:",
    layout=widgets.Layout(width="100%")
)

ytdlp_format = widgets.Text(value=DEFAULTS["ytdlp_format"], description="yt-dlp fmt:", layout=widgets.Layout(width="100%"))

drive_share_url = widgets.Text(
    value=DEFAULTS["drive_share_url"],
    placeholder="https://drive.google.com/file/d/.../view?usp=sharing",
    description="Lien Drive:",
    layout=widgets.Layout(width="100%")
)

conversion_mode = widgets.ToggleButtons(
    options=[("Preview", "preview"), ("Full", "full")],
    value=DEFAULTS["conversion_mode"],
    description="Mode:",
)

encoder = widgets.Dropdown(
    options=["vits", "vitb", "vitl", "vitg"],
    value=DEFAULTS["encoder"],
    description="ENCODER:"
)

sbs_w = widgets.IntText(value=DEFAULTS["sbs_w"], description="SBS W:")
sbs_h = widgets.IntText(value=DEFAULTS["sbs_h"], description="SBS H:")
max_shift = widgets.IntSlider(value=DEFAULTS["max_shift"], min=0, max=64, step=1, description="max_shift:")
alpha = widgets.FloatSlider(value=DEFAULTS["alpha"], min=0.0, max=1.0, step=0.01, description="alpha:")
input_size = widgets.IntText(value=DEFAULTS["input_size"], description="input_size:")
batch = widgets.IntText(value=DEFAULTS["batch"], description="batch:")

preview_interval = widgets.IntText(value=DEFAULTS["preview_interval"], description="preview_interval:")

keep_audio = widgets.Checkbox(value=DEFAULTS["keep_audio"], description="Garder l'audio")
crf = widgets.IntSlider(value=DEFAULTS["crf"], min=0, max=40, step=1, description="crf:")
preset = widgets.Dropdown(
    options=["ultrafast","veryfast","faster","fast","medium","slow","slower","veryslow"],
    value=DEFAULTS["preset"],
    description="preset:"
)

output_name_preview = widgets.Text(value=DEFAULTS["output_name_preview"], description="Sortie preview:")
use_drive = widgets.Checkbox(value=DEFAULTS["use_drive"], description="Monter Google Drive (optionnel)")

output_name_full = widgets.Text(value=DEFAULTS["output_name_full"], description="Sortie full:")

apply_btn = widgets.Button(description="Appliquer", button_style="success")
status = widgets.Output()

def _drive_file_id(url: str):
    """
    Extrait un file_id depuis différents formats de liens Drive.
    Exemples:
      https://drive.google.com/file/d/<ID>/view?usp=sharing
      https://drive.google.com/open?id=<ID>
      https://drive.google.com/uc?id=<ID>&export=download
    """
    if not url:
        return None
    m = re.search(r"/file/d/([a-zA-Z0-9_-]+)", url)
    if m:
        return m.group(1)
    m = re.search(r"[?&]id=([a-zA-Z0-9_-]+)", url)
    if m:
        return m.group(1)
    m = re.search(r"/uc\\?id=([a-zA-Z0-9_-]+)", url)
    if m:
        return m.group(1)
    return None

def _render_form():
    clear_output(wait=True)
    display(Markdown("## 📌 Configuration (Formulaire)"))
    display(source_type)

    if source_type.value == "yt-dlp":
        display(video_url)
        display(ytdlp_format)
    else:
        display(drive_share_url)

    display(conversion_mode)
    display(use_drive)
    display(Markdown("### Paramètres conversion"))
    display(widgets.HBox([encoder, preset]))
    display(widgets.HBox([sbs_w, sbs_h]))
    display(max_shift)
    display(alpha)
    display(widgets.HBox([input_size, batch]))
    display(Markdown("### Preview"))
    display(preview_interval)
    display(Markdown("### Encodage"))
    display(widgets.HBox([keep_audio, crf]))
    display(Markdown("### Noms de sortie"))
    display(widgets.VBox([output_name_preview, output_name_full]))
    display(apply_btn)
    display(status)

def _apply(_):
    with status:
        status.clear_output()
        # --- expose les variables globales ---
        globals()["SOURCE_TYPE"] = source_type.value
        globals()["VIDEO_URL"] = video_url.value.strip()
        globals()["YTDLP_FORMAT"] = ytdlp_format.value.strip() or "bv*+ba/b"
        globals()["DRIVE_SHARE_URL"] = drive_share_url.value.strip()
        globals()["CONVERSION_MODE"] = conversion_mode.value
        globals()["USE_DRIVE"] = bool(use_drive.value)

        globals()["ENCODER"] = encoder.value
        globals()["SBS_W"] = int(sbs_w.value)
        globals()["SBS_H"] = int(sbs_h.value)
        globals()["MAX_SHIFT"] = int(max_shift.value)
        globals()["ALPHA"] = float(alpha.value)
        globals()["INPUT_SIZE"] = int(input_size.value)
        globals()["BATCH"] = int(batch.value)

        globals()["PREVIEW_INTERVAL"] = int(preview_interval.value)

        globals()["KEEP_AUDIO"] = bool(keep_audio.value)
        globals()["CRF"] = int(crf.value)
        globals()["PRESET"] = preset.value

        globals()["OUTPUT_PREVIEW"] = output_name_preview.value.strip() or "preview_sbs.mp4"
        globals()["OUTPUT_FULL"] = output_name_full.value.strip() or "full_sbs.mp4"

        # Sorties: chemins "local" (disque Colab)
        globals()["OUTPUT_DIR_LOCAL"] = "/content"
        globals()["PREVIEW_LOCAL"] = f'{globals()["OUTPUT_DIR_LOCAL"]}/' + (globals()["OUTPUT_PREVIEW"].split("/")[-1])
        globals()["FULL_LOCAL"]    = f'{globals()["OUTPUT_DIR_LOCAL"]}/' + (globals()["OUTPUT_FULL"].split("/")[-1])

        # Sorties: chemins Drive (si USE_DRIVE)
        globals()["OUTPUT_DIR_DRIVE"] = "/content/drive/MyDrive/DepthAnythingSBS"
        globals()["PREVIEW_DRIVE"] = f'{globals()["OUTPUT_DIR_DRIVE"]}/' + (globals()["OUTPUT_PREVIEW"].split("/")[-1])
        globals()["FULL_DRIVE"]    = f'{globals()["OUTPUT_DIR_DRIVE"]}/' + (globals()["OUTPUT_FULL"].split("/")[-1])

        # Sortie active selon le mode (chemin local)
        globals()["OUTPUT_ACTIVE_LOCAL"] = globals()["PREVIEW_LOCAL"] if globals()["CONVERSION_MODE"]=="preview" else globals()["FULL_LOCAL"]
        # Sortie active selon le mode (chemin Drive)
        globals()["OUTPUT_ACTIVE_DRIVE"] = globals()["PREVIEW_DRIVE"] if globals()["CONVERSION_MODE"]=="preview" else globals()["FULL_DRIVE"]



        # chemins standard Colab
        globals()["WORKDIR"] = "/content"
        globals()["INPUT_VIDEO"] = "/content/input.mp4"
        globals()["INPUT_VIDEO_LOCAL"] = globals()["INPUT_VIDEO"]
        # validations
        errors = []
        if SOURCE_TYPE == "yt-dlp" and not VIDEO_URL:
            errors.append("• Tu as choisi yt-dlp mais l'URL est vide.")
        if SOURCE_TYPE == "drive_share":
            fid = _drive_file_id(DRIVE_SHARE_URL)
            globals()["DRIVE_FILE_ID"] = fid
            if not DRIVE_SHARE_URL or not fid:
                errors.append("• Tu as choisi Drive mais le lien est vide ou je n'arrive pas à extraire l'ID du fichier.")
        if CONVERSION_MODE not in ("preview","full"):
            errors.append("• Mode invalide (preview/full).")

        if errors:
            print("⚠️ Configuration incomplète :\\n" + "\\n".join(errors))
            return

        # résume
        print("✅ Configuration appliquée :")
        print(f"  SOURCE_TYPE      = {SOURCE_TYPE}")
        if SOURCE_TYPE == "yt-dlp":
            print(f"  VIDEO_URL        = {VIDEO_URL}")
            print(f"  YTDLP_FORMAT     = {YTDLP_FORMAT}")
        else:
            print(f"  DRIVE_FILE_ID    = {DRIVE_FILE_ID}")
        print(f"  CONVERSION_MODE  = {CONVERSION_MODE}")
        print(f"  USE_DRIVE        = {USE_DRIVE}")
        print(f"  ENCODER          = {ENCODER}")
        print(f"  SBS              = {SBS_W}x{SBS_H}")
        print(f"  max_shift/alpha  = {MAX_SHIFT} / {ALPHA}")
        print(f"  input_size/batch = {INPUT_SIZE} / {BATCH}")
        print(f"  preview_interval = {PREVIEW_INTERVAL}")
        print(f"  crf/preset       = {CRF} / {PRESET}")
        print(f"  outputs          = {OUTPUT_PREVIEW} ; {OUTPUT_FULL}")

apply_btn.on_click(_apply)

def _on_source_change(change):
    _render_form()

source_type.observe(_on_source_change, names="value")

_render_form()

In [ ]:
# (Optionnel) Vérification rapide : si tu n'as pas cliqué "Appliquer" dans la cellule CONFIG, ça stoppe ici.
required = ["USE_DRIVE", "SOURCE_TYPE","CONVERSION_MODE","ENCODER","SBS_W","SBS_H","MAX_SHIFT","ALPHA","INPUT_SIZE","BATCH","CRF","PRESET","INPUT_VIDEO","INPUT_VIDEO_LOCAL","OUTPUT_PREVIEW","OUTPUT_FULL"]
missing = [k for k in required if k not in globals()]
if missing:
    raise RuntimeError("Tu dois exécuter la cellule CONFIG et cliquer 'Appliquer' avant de continuer.\nManquants: " + ", ".join(missing))
print("✅ OK: config détectée")

In [ ]:
# 2) Compatibilité / Aliases globaux
# Cette cellule évite les NameError si certaines cellules attendent des anciennes variables.
def _ensure(name, value):
    if name not in globals():
        globals()[name] = value

# Entrées
_ensure("INPUT_VIDEO", "/content/input.mp4")
_ensure("INPUT_VIDEO_LOCAL", globals().get("INPUT_VIDEO", "/content/input.mp4"))

# Sorties (local)
_ensure("OUTPUT_DIR_LOCAL", "/content")
_ensure("OUTPUT_PREVIEW", "preview_sbs.mp4")
_ensure("OUTPUT_FULL", "full_sbs.mp4")
_ensure("PREVIEW_LOCAL", f"{OUTPUT_DIR_LOCAL}/" + OUTPUT_PREVIEW.split("/")[-1])
_ensure("FULL_LOCAL",    f"{OUTPUT_DIR_LOCAL}/" + OUTPUT_FULL.split("/")[-1])

# Sorties (drive)
_ensure("OUTPUT_DIR_DRIVE", "/content/drive/MyDrive/DepthAnythingSBS")
_ensure("PREVIEW_DRIVE", f"{OUTPUT_DIR_DRIVE}/" + OUTPUT_PREVIEW.split("/")[-1])
_ensure("FULL_DRIVE",    f"{OUTPUT_DIR_DRIVE}/" + OUTPUT_FULL.split("/")[-1])

# Sélection active
_ensure("CONVERSION_MODE", "preview")
_ensure("OUTPUT_ACTIVE_LOCAL", PREVIEW_LOCAL if CONVERSION_MODE=="preview" else FULL_LOCAL)
_ensure("OUTPUT_ACTIVE_DRIVE", PREVIEW_DRIVE if CONVERSION_MODE=="preview" else FULL_DRIVE)

# Flags
_ensure("USE_DRIVE", False)
_ensure("SOURCE_TYPE", "yt-dlp")
_ensure("VIDEO_URL", "")
_ensure("YTDLP_FORMAT", "bv*+ba/b")
_ensure("DRIVE_SHARE_URL", "")
_ensure("DRIVE_FILE_ID", None)

print("✅ Aliases globaux OK")

In [ ]:
# 1) Monter Google Drive (optionnel)
# Contrôlé par USE_DRIVE dans la cellule CONFIG.
if globals().get("USE_DRIVE", False):
    from google.colab import drive
    drive.mount('/content/drive')
else:
    print("USE_DRIVE=False -> pas de montage Google Drive")


In [ ]:
# 2) Vérifier le GPU
!nvidia-smi --query-gpu=name,memory.total --format=csv


In [ ]:
# Install yt-dlp (video downloader)
!pip -q install -U yt-dlp


In [ ]:
# =========================
# Télécharger via yt-dlp (garde le nom source, puis renomme en /content/input.mp4)
# =========================
import subprocess, pathlib, re

VIDEO_URL = (globals().get("VIDEO_URL","") or "").strip()
YTDLP_FORMAT = globals().get("YTDLP_FORMAT", "bv*+ba/b")

out_dir = pathlib.Path("/content")
out_dir.mkdir(parents=True, exist_ok=True)

if not VIDEO_URL:
    raise ValueError("VIDEO_URL vide.")

# Télécharge avec un vrai nom (titre + id) puis on renomme ensuite en input.mp4
tmpl = str(out_dir / "%(title).80s_%(id)s.%(ext)s")

cmd = [
    "yt-dlp",
    "-f", YTDLP_FORMAT,
    "--merge-output-format", "mp4",
    "-o", tmpl,
    VIDEO_URL,
]

print("Téléchargement via yt-dlp…")
print("Commande:", " ".join(cmd))
subprocess.check_call(cmd)

# dernier mp4 créé
mp4s = sorted(out_dir.glob("*.mp4"), key=lambda p: p.stat().st_mtime, reverse=True)
if not mp4s:
    raise RuntimeError("Aucun .mp4 trouvé après yt-dlp.")
downloaded = mp4s[0]

# ✅ on mémorise le nom d’origine (sans extension) pour la sortie
globals()["SOURCE_STEM"] = downloaded.stem

# puis on renomme en input.mp4 pour le script
input_path = out_dir / "input.mp4"
if downloaded.resolve() != input_path.resolve():
    downloaded.replace(input_path)

globals()["INPUT_VIDEO_LOCAL"] = str(input_path)
globals()["INPUT_VIDEO"] = str(input_path)

print("✅ Source originale :", globals()["SOURCE_STEM"])
print("✅ Entrée script    :", input_path)


In [ ]:
# 3) Cloner Depth-Anything-V2
import pathlib

WORK = pathlib.Path("/content")
REPO = WORK / "Depth-Anything-V2"

if not REPO.exists():
    !git clone https://github.com/DepthAnything/Depth-Anything-V2.git
else:
    print("Repo déjà présent :", REPO)

%cd /content/Depth-Anything-V2
!ls -la | head


In [ ]:
# 4) Installer les dépendances (Colab-friendly, minimal pour ton script)
# ⚠️ On évite requirements.txt ici (souvent source de conflits numpy dans Colab)
!pip -q install -U pip
!pip -q install -U "numpy>=2.0,<2.2" tqdm opencv-python-headless

# Vérifs
!python -c "import numpy as np, cv2, torch; print('numpy', np.__version__); print('cv2', cv2.__version__); print('torch', torch.__version__); print('cuda', torch.cuda.is_available())"
!which ffmpeg && ffmpeg -version | head -n 2
!ffmpeg -hide_banner -encoders | grep -E "h264_nvenc|hevc_nvenc" || true


In [ ]:
# =========================
# Télécharger le script mono_to_sbs depuis GitHub (ou autre URL)
# =========================
import pathlib, os, re, urllib.request

# Fallbacks robustes si tu n'as pas (encore) exécuté la cellule CONFIG
SCRIPT_URL = globals().get(
    "SCRIPT_URL",
    "https://github.com/hashtag1138/Depth-Anything-collab-notebook/blob/main/mono_to_sbs_pico4_v2.py",
)
SCRIPT_TARGET = globals().get(
    "SCRIPT_TARGET",
    "/content/Depth-Anything-V2/mono_to_sbs_pico4_v2.py",
)

script_url = (SCRIPT_URL or "").strip()
target = pathlib.Path(SCRIPT_TARGET)

if not script_url:
    raise ValueError("SCRIPT_URL est vide. Mets une URL ou exécute la cellule CONFIG puis clique 'Appliquer'.")

target.parent.mkdir(parents=True, exist_ok=True)

def github_blob_to_raw(url: str) -> str:
    """
    Convertit:
      https://github.com/<user>/<repo>/blob/<branch>/path/file.py
    en:
      https://raw.githubusercontent.com/<user>/<repo>/<branch>/path/file.py
    """
    m = re.match(r"^https?://github\.com/([^/]+)/([^/]+)/blob/([^/]+)/(.*)$", url)
    if not m:
        return url
    user, repo, branch, path = m.groups()
    return f"https://raw.githubusercontent.com/{user}/{repo}/{branch}/{path}"

raw_url = github_blob_to_raw(script_url)

print("Téléchargement du script...")
print("URL:", script_url)
print("RAW:", raw_url)
print("Dest:", target)

# Télécharger (urllib pour éviter dépendances)
req = urllib.request.Request(raw_url, headers={"User-Agent": "Mozilla/5.0"})
with urllib.request.urlopen(req) as r:
    data = r.read()

# Sanity check : éviter de sauvegarder une page HTML à la place du .py
head = data[:200].lower()
if b"<html" in head or b"<!doctype" in head:
    raise RuntimeError(
        "Le contenu téléchargé ressemble à du HTML (pas un .py). "
        "Vérifie l'URL (elle doit être 'raw')."
    )

target.write_bytes(data)

# Petite vérif : taille et aperçu
print(f"OK ✅ écrit {target} ({target.stat().st_size/1024:.1f} KB)")
print("Aperçu (10 premières lignes):")
print("-" * 60)
with target.open("r", encoding="utf-8", errors="replace") as f:
    for _ in range(10):
        line = f.readline()
        if not line:
            break
        print(line.rstrip())
print("-" * 60)


In [ ]:
# 6) Télécharger le checkpoint Depth-Anything-V2 depuis les liens du README (Hugging Face)
# Les modèles "relatifs" du README :
# - vits : Depth-Anything-V2-Small  -> depth_anything_v2_vits.pth
# - vitb : Depth-Anything-V2-Base   -> depth_anything_v2_vitb.pth
# - vitl : Depth-Anything-V2-Large  -> depth_anything_v2_vitl.pth
#
# Note: le vitg (Giant) peut ne pas être publié selon le README; on gère ce cas proprement.

import os, pathlib, urllib.request

ENCODER = "vits"  # vits / vitb / vitl

URLS = {
    "vits": "https://huggingface.co/depth-anything/Depth-Anything-V2-Small/resolve/main/depth_anything_v2_vits.pth?download=true",
    "vitb": "https://huggingface.co/depth-anything/Depth-Anything-V2-Base/resolve/main/depth_anything_v2_vitb.pth?download=true",
    "vitl": "https://huggingface.co/depth-anything/Depth-Anything-V2-Large/resolve/main/depth_anything_v2_vitl.pth?download=true",
}

if ENCODER not in URLS:
    raise ValueError(
        f"ENCODER={ENCODER!r} non supporté ici. Choisis parmi: {list(URLS)} "
        "(le vitg/giant n'est pas listé comme checkpoint 'Download' dans le README)."
    )

REPO = pathlib.Path("/content/Depth-Anything-V2")
CKPT_DIR = REPO / "checkpoints"
CKPT_DIR.mkdir(parents=True, exist_ok=True)

ckpt_path = CKPT_DIR / f"depth_anything_v2_{ENCODER}.pth"
url = URLS[ENCODER]

def download(url: str, dst: pathlib.Path):
    print("Téléchargement:", url)
    print("->", dst)
    # urllib.request.urlretrieve affiche un progress si on fournit un hook
    def report(count, block_size, total_size):
        if total_size <= 0:
            return
        downloaded = count * block_size
        pct = min(100.0, downloaded * 100.0 / total_size)
        print(f"  {pct:5.1f}% ({downloaded/1e6:.1f} / {total_size/1e6:.1f} MB)", end="\r")
    urllib.request.urlretrieve(url, dst.as_posix(), reporthook=report)
    print("\nOK.")

if ckpt_path.exists() and ckpt_path.stat().st_size > 10_000_000:
    print("Checkpoint déjà présent :", ckpt_path, f"({ckpt_path.stat().st_size/1e6:.1f} MB)")
else:
    if ckpt_path.exists():
        ckpt_path.unlink()
    download(url, ckpt_path)

print("Contenu checkpoints/:")
!ls -lh checkpoints | head -n 20


In [ ]:
# =========================
# Détecter le fichier d'entrée (local / Drive)
# =========================
import pathlib

# Fallbacks (au cas où CONFIG n'a pas été appliquée)
USE_DRIVE = bool(globals().get("USE_DRIVE", False))

INPUT_VIDEO_LOCAL = globals().get("INPUT_VIDEO_LOCAL", "/content/input.mp4")
# Optionnel: si tu veux forcer un chemin Drive, mets-le dans CONFIG, sinon fallback
INPUT_VIDEO_DRIVE = globals().get("INPUT_VIDEO_DRIVE", "/content/drive/MyDrive/input.mp4")

candidates = []

# a) Local d'abord
if INPUT_VIDEO_LOCAL:
    candidates.append(pathlib.Path(INPUT_VIDEO_LOCAL))

# b) sinon Drive (si activé)
if USE_DRIVE and INPUT_VIDEO_DRIVE:
    candidates.append(pathlib.Path(INPUT_VIDEO_DRIVE))

# Choisir le premier qui existe
INPUT_VIDEO = None
for p in candidates:
    if p.exists() and p.is_file() and p.stat().st_size > 0:
        INPUT_VIDEO = str(p)
        break

if not INPUT_VIDEO:
    print("❌ Aucun fichier vidéo trouvé.")
    print("Candidats testés :")
    for p in candidates:
        print(" -", p)
    raise FileNotFoundError(
        "Aucun input vidéo détecté. "
        "Télécharge la vidéo (cellule yt-dlp/drive_share) ou mets INPUT_VIDEO_LOCAL/INPUT_VIDEO_DRIVE dans CONFIG."
    )

print("✅ INPUT_VIDEO =", INPUT_VIDEO)


In [ ]:
# =========================
# Sortie: utiliser le nom source (SOURCE_STEM) + _sbs.mp4
# =========================
from pathlib import Path

mode = globals().get("CONVERSION_MODE", "full")  # preview/full

# si SOURCE_STEM n’existe pas (cas rare), fallback sur input.mp4
stem = globals().get("SOURCE_STEM") or Path(globals().get("INPUT_VIDEO","/content/input.mp4")).stem

# Ton requirement : full => <stem>_sbs.mp4
# (et preview pour éviter d’écraser le full)
out_name = f"{stem}_sbs.mp4" if mode == "full" else f"{stem}_preview_sbs.mp4"

OUTPUT_DIR_LOCAL = globals().get("OUTPUT_DIR_LOCAL", "/content")
OUTPUT_DIR_DRIVE = globals().get("OUTPUT_DIR_DRIVE", "/content/drive/MyDrive/DepthAnythingSBS")

globals()["OUTPUT_ACTIVE_LOCAL"] = str(Path(OUTPUT_DIR_LOCAL) / out_name)
globals()["OUTPUT_ACTIVE_DRIVE"] = str(Path(OUTPUT_DIR_DRIVE) / out_name)

print("✅ SOURCE_STEM =", stem)
print("✅ OUTPUT_ACTIVE_LOCAL =", globals()["OUTPUT_ACTIVE_LOCAL"])
print("✅ OUTPUT_ACTIVE_DRIVE =", globals()["OUTPUT_ACTIVE_DRIVE"])


In [ ]:
import os, sys, shlex, subprocess
from pathlib import Path

# ---- trouver le script ----
candidates = []
if globals().get("SCRIPT_TARGET"):
    candidates.append(Path(globals()["SCRIPT_TARGET"]))
candidates += [
    Path("/content/Depth-Anything-V2/mono_to_sbs_pico4_v2.py"),
    Path("/content/mono_to_sbs_pico4_v2.py"),
    Path.cwd() / "mono_to_sbs_pico4_v2.py",
]
script_path = next((p for p in candidates if p.exists() and p.is_file() and p.stat().st_size > 0), None)
if not script_path:
    print("❌ Script introuvable. Chemins testés :")
    for p in candidates: print(" -", p)
    raise FileNotFoundError("mono_to_sbs_pico4_v2.py introuvable.")

# se placer dans le dossier du script
os.chdir(str(script_path.parent))
print("✅ Script:", script_path)
print("📂 CWD:", os.getcwd())

# ---- input/output ----
inp = globals().get("INPUT_VIDEO") or globals().get("INPUT_VIDEO_LOCAL") or "/content/input.mp4"
out = globals().get("OUTPUT_ACTIVE_LOCAL") or globals().get("FULL_LOCAL") or "/content/full_sbs.mp4"

# ---- paramètres du script ----
ENCODER = globals().get("ENCODER","vits")
SBS_W = int(globals().get("SBS_W",2560))
SBS_H = int(globals().get("SBS_H",1440))
MAX_SHIFT = int(globals().get("MAX_SHIFT",24))
ALPHA = float(globals().get("ALPHA",0.90))
INPUT_SIZE = int(globals().get("INPUT_SIZE",518))
BATCH = int(globals().get("BATCH",52))

CONVERSION_MODE = globals().get("CONVERSION_MODE","full")
PREVIEW_INTERVAL = int(globals().get("PREVIEW_INTERVAL",120))
KEEP_AUDIO = bool(globals().get("KEEP_AUDIO", True))

# Qualité / encode: TON SCRIPT utilise CQ + nv_preset
CQ = int(globals().get("CQ", 18))                 # qualité (plus bas = meilleure qualité, fichier plus gros)
NV_PRESET = globals().get("NV_PRESET", "slow")    # preset nvenc (si ton script le mappe)

# Optionnel : fps override
FPS = globals().get("FPS", None)                  # ex: 30, 60, etc.

# ---- commande ----
cmd = [
    sys.executable, "-u", str(script_path),
    inp, out,
    "--encoder", str(ENCODER),
    "--sbs_w", str(SBS_W),
    "--sbs_h", str(SBS_H),
    "--max_shift", str(MAX_SHIFT),
    "--alpha", str(ALPHA),
    "--input_size", str(INPUT_SIZE),
    "--batch", str(BATCH),
    "--cq", str(CQ),
    "--nv_preset", str(NV_PRESET),
]

if FPS is not None:
    cmd += ["--fps", str(FPS)]

if CONVERSION_MODE == "preview":
    cmd += ["--preview", "--preview_interval", str(PREVIEW_INTERVAL)]

if not KEEP_AUDIO:
    cmd += ["--no_audio"]

print("Commande:\n", " ".join(shlex.quote(x) for x in cmd))

# ---- exécution logs live ----
p = subprocess.Popen(cmd, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True, bufsize=1)
for line in p.stdout:
    print(line, end="")
ret = p.wait()
if ret != 0:
    raise RuntimeError(f"Conversion failed (exit code {ret})")

print("\n✅ OK:", out)


In [ ]:
# =========================
# Copier la sortie active vers Drive
# =========================
import pathlib

src = pathlib.Path(globals().get("OUTPUT_ACTIVE_LOCAL", "/content/full_sbs.mp4"))
dst = pathlib.Path(globals().get("OUTPUT_ACTIVE_DRIVE", "/content/drive/MyDrive/DepthAnythingSBS/full_sbs.mp4"))

dst.parent.mkdir(parents=True, exist_ok=True)

!rsync -ah --info=progress2 "{src}" "{dst}"
print("✅ Copié sur Drive:", dst)

## Notes perf
- Sur **T4 16GB**, si le FULL est trop lent :
  - `--input_size 448` (voire 384)
  - `--preset veryfast` + `--crf 20–22`
- Tu as NVENC disponible (`h264_nvenc`). Pour l’exploiter, il faut patcher `encode_video()` (libx264 → h264_nvenc) dans ton script.
